In [11]:
#ipynb 파일 불러오기 위해서
#!pip install import_ipynb
import import_ipynb

In [15]:
from opponentsLOL import RiotApi


class opponentsParser(RiotApi):
    """
    AUTHOR:이종법
    DATE:2021-03-30
    PURPOSE: RiotApi를 상속받아서, 해당 내용을 parsing하는 class
    해당 클래스에서는 requests를 따로 안써도된다.
    
    
    """

    
    def __init__(self,apikey):
        super().__init__(apikey)
        
        print(self._apikey)
        
    def timeline_to_items(self,match_ids):
        
        #match_id 들이 담긴 list에서 각각 타임라인을 호출한다.
        #특정 타임라인에서 item 구매 이벤트들을 저장한다.
        
        
        pass

    def timeline_to_skills(self,timeline_data):
        #특정 타임라인에서 skill levelup 이벤트들을 저장한다.
        
        pass    
    

    def get_match_ids_test(self):
        # return [match_ids]
        # test를 위해 작성된 코드. file을 읽어옴.
        # file을 읽어서 account_id를 받아서. game_id를 받고. set로 game_id 중복을 제거하여 출력해준다.
        
        game_id_set = set() # game_id를 받아서, set에 모두 담아준다. 중복제거됨.
        
        f = open("./resource/getAccountId.txt", 'r',encoding='utf-8')
        test_index = 1
        while True:
            line = f.readline().strip()
            if not line: break
            print(test_index,"처리중")
            print(line)
            temp_data =self.get_gameid_byAccountid(line)
            game_id_set.update(temp_data)
            test_index=test_index+1
        f.close()
        print(game_id_set)
        #with open("/resource/foo.txt", "w") as f:
        #    f.write("Life is too short, you need python")
        
        


    
#단위 테스트
if __name__ == '__main__':
    test = opponentsParser('RGAPI-1db9d2d1-76ed-4991-9162-471b0b0cd030')
    
    games_ids=test.get_match_ids_test()

RiotApi init
RGAPI-1db9d2d1-76ed-4991-9162-471b0b0cd030
1 처리중
XnR2btHq6sv15Kut6qmgSc5aiQRhfCRANTaQyvksETm1m2NEiHfFPik_
https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/XnR2btHq6sv15Kut6qmgSc5aiQRhfCRANTaQyvksETm1m2NEiHfFPik_?api_key=RGAPI-1db9d2d1-76ed-4991-9162-471b0b0cd030
{'matches': [{'platformId': 'KR', 'gameId': 5100271040, 'champion': 112, 'queue': 420, 'season': 13, 'timestamp': 1617095697364, 'role': 'DUO_SUPPORT', 'lane': 'NONE'}, {'platformId': 'KR', 'gameId': 5100118302, 'champion': 142, 'queue': 420, 'season': 13, 'timestamp': 1617093780885, 'role': 'SOLO', 'lane': 'MID'}, {'platformId': 'KR', 'gameId': 5100105839, 'champion': 429, 'queue': 420, 'season': 13, 'timestamp': 1617092333206, 'role': 'DUO_SUPPORT', 'lane': 'NONE'}, {'platformId': 'KR', 'gameId': 5100171561, 'champion': 517, 'queue': 420, 'season': 13, 'timestamp': 1617090677242, 'role': 'SOLO', 'lane': 'MID'}, {'platformId': 'KR', 'gameId': 5099467540, 'champion': 517, 'queue': 420, 'season': 13, 

{'matches': [{'platformId': 'KR', 'gameId': 5100196092, 'champion': 246, 'queue': 420, 'season': 13, 'timestamp': 1617093775842, 'role': 'SOLO', 'lane': 'MID'}, {'platformId': 'KR', 'gameId': 5099956583, 'champion': 91, 'queue': 420, 'season': 13, 'timestamp': 1617081360709, 'role': 'DUO_SUPPORT', 'lane': 'NONE'}, {'platformId': 'KR', 'gameId': 5099742991, 'champion': 4, 'queue': 420, 'season': 13, 'timestamp': 1617050763712, 'role': 'DUO_SUPPORT', 'lane': 'NONE'}, {'platformId': 'KR', 'gameId': 5099712844, 'champion': 91, 'queue': 420, 'season': 13, 'timestamp': 1617048667110, 'role': 'DUO', 'lane': 'MID'}, {'platformId': 'KR', 'gameId': 5099680703, 'champion': 4, 'queue': 420, 'season': 13, 'timestamp': 1617044108861, 'role': 'SOLO', 'lane': 'MID'}, {'platformId': 'KR', 'gameId': 5099506214, 'champion': 4, 'queue': 420, 'season': 13, 'timestamp': 1617039780710, 'role': 'SOLO', 'lane': 'MID'}, {'platformId': 'KR', 'gameId': 5099336432, 'champion': 3, 'queue': 420, 'season': 13, 'times

{'matches': [{'platformId': 'KR', 'gameId': 5099500285, 'champion': 876, 'queue': 420, 'season': 13, 'timestamp': 1617035358213, 'role': 'DUO_SUPPORT', 'lane': 'NONE'}, {'platformId': 'KR', 'gameId': 5099317988, 'champion': 876, 'queue': 420, 'season': 13, 'timestamp': 1617033817816, 'role': 'NONE', 'lane': 'JUNGLE'}, {'platformId': 'KR', 'gameId': 5099260164, 'champion': 201, 'queue': 420, 'season': 13, 'timestamp': 1617030129383, 'role': 'DUO_SUPPORT', 'lane': 'BOTTOM'}, {'platformId': 'KR', 'gameId': 5099180778, 'champion': 876, 'queue': 420, 'season': 13, 'timestamp': 1617027204672, 'role': 'NONE', 'lane': 'JUNGLE'}, {'platformId': 'KR', 'gameId': 5099065222, 'champion': 876, 'queue': 420, 'season': 13, 'timestamp': 1617025417861, 'role': 'NONE', 'lane': 'JUNGLE'}, {'platformId': 'KR', 'gameId': 5098958443, 'champion': 120, 'queue': 420, 'season': 13, 'timestamp': 1617023394409, 'role': 'NONE', 'lane': 'JUNGLE'}, {'platformId': 'KR', 'gameId': 5098961355, 'champion': 876, 'queue': 

{'matches': [{'platformId': 'KR', 'gameId': 5099272884, 'champion': 145, 'queue': 420, 'season': 13, 'timestamp': 1617031293407, 'role': 'DUO_CARRY', 'lane': 'BOTTOM'}, {'platformId': 'KR', 'gameId': 5099106398, 'champion': 360, 'queue': 420, 'season': 13, 'timestamp': 1617028707430, 'role': 'DUO_CARRY', 'lane': 'BOTTOM'}, {'platformId': 'KR', 'gameId': 5099054219, 'champion': 360, 'queue': 420, 'season': 13, 'timestamp': 1617025090252, 'role': 'DUO_CARRY', 'lane': 'BOTTOM'}, {'platformId': 'KR', 'gameId': 5098995867, 'champion': 12, 'queue': 420, 'season': 13, 'timestamp': 1617023111145, 'role': 'DUO_SUPPORT', 'lane': 'BOTTOM'}, {'platformId': 'KR', 'gameId': 5098921129, 'champion': 360, 'queue': 420, 'season': 13, 'timestamp': 1617021109420, 'role': 'DUO_CARRY', 'lane': 'BOTTOM'}, {'platformId': 'KR', 'gameId': 5098773866, 'champion': 145, 'queue': 420, 'season': 13, 'timestamp': 1617015399557, 'role': 'DUO_CARRY', 'lane': 'BOTTOM'}, {'platformId': 'KR', 'gameId': 5098688885, 'champi

{'matches': [{'platformId': 'KR', 'gameId': 5100217603, 'champion': 76, 'queue': 420, 'season': 13, 'timestamp': 1617098637543, 'role': 'NONE', 'lane': 'JUNGLE'}, {'platformId': 'KR', 'gameId': 5100213973, 'champion': 2, 'queue': 420, 'season': 13, 'timestamp': 1617096683749, 'role': 'NONE', 'lane': 'JUNGLE'}, {'platformId': 'KR', 'gameId': 5100137483, 'champion': 2, 'queue': 420, 'season': 13, 'timestamp': 1617093572210, 'role': 'NONE', 'lane': 'JUNGLE'}, {'platformId': 'KR', 'gameId': 5099722296, 'champion': 76, 'queue': 420, 'season': 13, 'timestamp': 1617047967027, 'role': 'DUO_SUPPORT', 'lane': 'NONE'}, {'platformId': 'KR', 'gameId': 5099740933, 'champion': 76, 'queue': 420, 'season': 13, 'timestamp': 1617045924311, 'role': 'NONE', 'lane': 'JUNGLE'}, {'platformId': 'KR', 'gameId': 5099640605, 'champion': 2, 'queue': 420, 'season': 13, 'timestamp': 1617043265695, 'role': 'NONE', 'lane': 'JUNGLE'}, {'platformId': 'KR', 'gameId': 5099467902, 'champion': 120, 'queue': 420, 'season': 1

...

{'matches': [{'platformId': 'KR', 'gameId': 5099935346, 'champion': 57, 'queue': 420, 'season': 13, 'timestamp': 1617080193132, 'role': 'DUO_SUPPORT', 'lane': 'NONE'}, {'platformId': 'KR', 'gameId': 5099913071, 'champion': 43, 'queue': 420, 'season': 13, 'timestamp': 1617078316667, 'role': 'DUO_SUPPORT', 'lane': 'BOTTOM'}, {'platformId': 'KR', 'gameId': 5099960095, 'champion': 147, 'queue': 420, 'season': 13, 'timestamp': 1617076519380, 'role': 'DUO_SUPPORT', 'lane': 'BOTTOM'}, {'platformId': 'KR', 'gameId': 5099808535, 'champion': 80, 'queue': 420, 'season': 13, 'timestamp': 1617074291132, 'role': 'DUO_SUPPORT', 'lane': 'BOTTOM'}, {'platformId': 'KR', 'gameId': 5099893807, 'champion': 526, 'queue': 420, 'season': 13, 'timestamp': 1617072358214, 'role': 'DUO_SUPPORT', 'lane': 'NONE'}, {'platformId': 'KR', 'gameId': 5099852695, 'champion': 57, 'queue': 420, 'season': 13, 'timestamp': 1617069578461, 'role': 'DUO_SUPPORT', 'lane': 'BOTTOM'}, {'platformId': 'KR', 'gameId': 5099783935, 'cha

{'matches': [{'platformId': 'KR', 'gameId': 5100197428, 'champion': 18, 'queue': 420, 'season': 13, 'timestamp': 1617094442113, 'role': 'DUO_CARRY', 'lane': 'BOTTOM'}, {'platformId': 'KR', 'gameId': 5100144835, 'champion': 51, 'queue': 420, 'season': 13, 'timestamp': 1617092116888, 'role': 'DUO_CARRY', 'lane': 'BOTTOM'}, {'platformId': 'KR', 'gameId': 5100112274, 'champion': 145, 'queue': 420, 'season': 13, 'timestamp': 1617090516834, 'role': 'DUO_CARRY', 'lane': 'BOTTOM'}, {'platformId': 'KR', 'gameId': 5100029894, 'champion': 81, 'queue': 420, 'season': 13, 'timestamp': 1617089338132, 'role': 'DUO', 'lane': 'NONE'}, {'platformId': 'KR', 'gameId': 5100056449, 'champion': 81, 'queue': 420, 'season': 13, 'timestamp': 1617087710887, 'role': 'DUO_CARRY', 'lane': 'BOTTOM'}, {'platformId': 'KR', 'gameId': 5100044707, 'champion': 18, 'queue': 420, 'season': 13, 'timestamp': 1617086433745, 'role': 'DUO_SUPPORT', 'lane': 'NONE'}, {'platformId': 'KR', 'gameId': 5100030200, 'champion': 145, 'que

{'matches': [{'platformId': 'KR', 'gameId': 5100481508, 'champion': 79, 'queue': 420, 'season': 13, 'timestamp': 1617101067876, 'role': 'NONE', 'lane': 'JUNGLE'}, {'platformId': 'KR', 'gameId': 5100267995, 'champion': 120, 'queue': 420, 'season': 13, 'timestamp': 1617099155204, 'role': 'DUO_SUPPORT', 'lane': 'NONE'}, {'platformId': 'KR', 'gameId': 5100245290, 'champion': 79, 'queue': 420, 'season': 13, 'timestamp': 1617097686506, 'role': 'DUO_SUPPORT', 'lane': 'NONE'}, {'platformId': 'KR', 'gameId': 5100250322, 'champion': 120, 'queue': 420, 'season': 13, 'timestamp': 1617095092024, 'role': 'NONE', 'lane': 'JUNGLE'}, {'platformId': 'KR', 'gameId': 5100155578, 'champion': 120, 'queue': 420, 'season': 13, 'timestamp': 1617092659716, 'role': 'NONE', 'lane': 'JUNGLE'}, {'platformId': 'KR', 'gameId': 5100122962, 'champion': 120, 'queue': 420, 'season': 13, 'timestamp': 1617091006602, 'role': 'NONE', 'lane': 'JUNGLE'}, {'platformId': 'KR', 'gameId': 5100140030, 'champion': 120, 'queue': 420,

{'matches': [{'platformId': 'KR', 'gameId': 5100030200, 'champion': 120, 'queue': 420, 'season': 13, 'timestamp': 1617083602179, 'role': 'NONE', 'lane': 'JUNGLE'}, {'platformId': 'KR', 'gameId': 5099947584, 'champion': 203, 'queue': 420, 'season': 13, 'timestamp': 1617081834784, 'role': 'NONE', 'lane': 'JUNGLE'}, {'platformId': 'KR', 'gameId': 5099964706, 'champion': 203, 'queue': 420, 'season': 13, 'timestamp': 1617080111015, 'role': 'NONE', 'lane': 'JUNGLE'}, {'platformId': 'KR', 'gameId': 5099899934, 'champion': 876, 'queue': 420, 'season': 13, 'timestamp': 1617077607909, 'role': 'NONE', 'lane': 'JUNGLE'}, {'platformId': 'KR', 'gameId': 5099857746, 'champion': 163, 'queue': 420, 'season': 13, 'timestamp': 1617074392185, 'role': 'NONE', 'lane': 'JUNGLE'}, {'platformId': 'KR', 'gameId': 5099874683, 'champion': 121, 'queue': 420, 'season': 13, 'timestamp': 1617071991996, 'role': 'NONE', 'lane': 'JUNGLE'}, {'platformId': 'KR', 'gameId': 5099804314, 'champion': 121, 'queue': 420, 'season

{'matches': [{'platformId': 'KR', 'gameId': 5100411712, 'champion': 81, 'queue': 420, 'season': 13, 'timestamp': 1617100735285, 'role': 'DUO_CARRY', 'lane': 'BOTTOM'}, {'platformId': 'KR', 'gameId': 5100236956, 'champion': 145, 'queue': 420, 'season': 13, 'timestamp': 1617098461053, 'role': 'DUO_CARRY', 'lane': 'BOTTOM'}, {'platformId': 'KR', 'gameId': 5100310669, 'champion': 145, 'queue': 420, 'season': 13, 'timestamp': 1617096199453, 'role': 'DUO', 'lane': 'BOTTOM'}, {'platformId': 'KR', 'gameId': 5099742265, 'champion': 145, 'queue': 420, 'season': 13, 'timestamp': 1617048776864, 'role': 'DUO_SUPPORT', 'lane': 'NONE'}, {'platformId': 'KR', 'gameId': 5099721545, 'champion': 12, 'queue': 420, 'season': 13, 'timestamp': 1617046271395, 'role': 'DUO_SUPPORT', 'lane': 'BOTTOM'}, {'platformId': 'KR', 'gameId': 5099710581, 'champion': 81, 'queue': 420, 'season': 13, 'timestamp': 1617044080815, 'role': 'DUO_CARRY', 'lane': 'BOTTOM'}, {'platformId': 'KR', 'gameId': 5099419603, 'champion': 145